# Run MoPro from QCrBox

## Import and setting up folders / paths

In [1]:
import shutil

from qcrbox_wrapper import QCrBoxPathHelper, QCrBoxWrapper

We create a path to run our MoPro example and get our paths from our own computer's filesystem and the internal filesystem of the QCrBox containers.

Using the dotenv package makes things more convenient as we can read the environment variables from the .env.dev file in the QCrBox directory. If you want to run with python core packages only, use the `__init__` method instead by defining the path to the shared directory explicitely in `path_to_shared_dir` and replacing the next four lines with:

```python
pathhelper = QCrBoxPathHelper(
    path_to_shared_dir,
    'examples_crystal_explorer'
)
```

In [2]:
pathhelper = QCrBoxPathHelper.from_dotenv(".env.dev", "examples_mopro")

path_local = pathhelper.local_path
path_qcrbox = pathhelper.qcrbox_path

## Connecting to QCrBox

We can connect to the QCrBox Inventory via python after we have started everything with `qcb up`. Sometimes the server takes a while so you might need to retry if it initially refuses connection. This should not take more than 30 seconds after your console output says that everything has started.

In [3]:
qcrbox = QCrBoxWrapper.from_server_addr("127.0.0.1", 11000)

We should see `Crystalexplorer in our application_dict and should put it into a variable.

In [ ]:
qcrbox.application_dict

In [ ]:
mopro = qcrbox.application_dict["MoProSuite"]

## Running the MoPro GUI in QCrBox

We can now access the GUI of MoPro within the container. To run the command needs a unified cif, which is then internally converted into the format CrystalExplorer needs. For more read the Olex2 Example Notebook (TODO Write and link a more complete explanation in docs). 

Once we run the `interactive` command, two things happen: Firstly, a new browser window with the GUI opens up. Secondly, you should see an input prompt in your execution engine for this notebook. (In Jupyter this should be at the end of the cell, in VSCode it is at the top). By pressing enter within this prompt we tell QCrBox that we are done. There is no back conversion, as we do not expect MoPro to modify the cif file. Note that this input prompt is a stopgap for an actual UI, so this feels a bit clunky at the moment.

In [ ]:
interact_local_folder, interact_qcrbox_folder = pathhelper.path_to_pair('interactive')
interact_local_folder.mkdir(exist_ok=True, parents=True)

shutil.copy('./input_files/input.cif', interact_local_folder / 'input.cif')

In [ ]:
session = mopro.interactive_session(
    input_cif_path=interact_qcrbox_folder / 'input.cif',
    output_cif_path=interact_qcrbox_folder / 'output.cif',
    table_type='XD',
    wavefunction_type='Su Coppens'
)
session.start()

In [ ]:
session.close()

In [7]:
run_inp_local_folder, run_inp_qcrbox_folder = pathhelper.path_to_pair('run_inp_file')
run_inp_local_folder.mkdir(exist_ok=True)

In [8]:
shutil.copy('./input_files/input.cif', run_inp_local_folder / 'input.cif')

calc = mopro.run_inp_file(
    input_cif_path=run_inp_qcrbox_folder / 'input.cif',
    output_cif_path=run_inp_qcrbox_folder / 'output.cif',
    inp_file_path= interact_qcrbox_folder / 'mopro_guide.inp', # notice that we use the inp file from the interactive calculation
    constraint_file_path=None,
    restraint_file_path=None,
    table_type='XD',
    wavefunction_type='Su Coppens'
)

In [ ]:
calc.wait_while_running(1.0)